# Gapfilled timeserie

In [2]:
import glob, os
import numpy as np
import pandas as pd
import rasterio
from pathlib import Path
from IPython.display import display
import plotly.express as px
import plotly.graph_objects as go
import scipy as sp
import scipy.sparse
import scipy.linalg
from scipy.sparse.linalg import cg

print('All libraries successfully imported!')
print(f'Scipy : {sp.__version__}')

All libraries successfully imported!
Scipy : 1.3.3


## Set directory

In [2]:
computer_path = '/export/miro/ndeffense/LBRAT2104/'
grp_letter    = 'X'

# Directory for all work files
work_path = f'{computer_path}GROUP_{grp_letter}/WORK/'

im_path = f'{work_path}NDVI/'

## Stack all raster into a single 3-D array

In [3]:
list_im = sorted(glob.glob(f'{im_path}*.tif'))

for im_file in list_im:
    print(im_file)

/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200116T105309_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200212T104049_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200316T104709_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200417T104021_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200520T105031_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200621T103629_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200719T105031_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200813T104629_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20200914T104031_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20201019T103959_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20201118T104329_NDVI.tif
/export/miro/ndeffense/LBRAT2104/GROUP_X/WORK/NDVI/T31UFS_20201218T104349_NDVI.tif


In [4]:
list_dates = []

for im_file in list_im:
    date = os.path.basename(im_file)[7:7+8]

    list_dates.append(date)

print(list_dates)

['20200116', '20200212', '20200316', '20200417', '20200520', '20200621', '20200719', '20200813', '20200914', '20201019', '20201118', '20201218']


In [5]:
list_src_arr = []

for im_file in list_im:

    src = rasterio.open(im_file, "r")
    im = src.read(1)
    list_src_arr.append(im)
    src.close()


ts_arr = np.stack(list_src_arr, axis=0)

print(ts_arr)
print(ts_arr.shape)

[[[0.66026365 0.50760563 0.40082079 ... 0.83809524 0.85211103 0.79693487]
  [0.71828172 0.64799294 0.59344262 ... 0.84845606 0.82850356 0.65788801]
  [0.7516672  0.67531083 0.63619575 ... 0.81905001 0.65665236 0.42105263]
  ...
  [0.59484778 0.55236576 0.55077539 ...        nan 0.24806746 0.31261561]
  [0.60609911 0.56308411 0.55648536 ...        nan 0.24914676 0.25447244]
  [       nan        nan        nan ...        nan        nan        nan]]

 [[0.42912925 0.43237788 0.37547408 ...        nan        nan        nan]
  [0.43097239 0.43376623 0.42655262 ...        nan        nan        nan]
  [0.43283582 0.44285215 0.44069015 ...        nan        nan        nan]
  ...
  [       nan        nan        nan ...        nan        nan        nan]
  [       nan        nan        nan ...        nan        nan        nan]
  [       nan        nan        nan ...        nan        nan        nan]]

 [[0.66888396 0.59018987 0.47197107 ... 0.61890517 0.60756501 0.6372093 ]
  [0.69617772 0.651455

## Easy test

In [39]:
date_1 = np.array([[1, 2, 3],
                  [4, 5, np.nan],
                  [3, 7, 9]])

date_2 = np.array([[np.nan, 2, 3],
                  [4, 5, 10],
                  [np.nan, 7, 9]])

date_3 = np.array([[3, 2, 3],
                  [9, 10, np.nan],
                  [5, 7, 9]])

list_src_arr = [date_1, date_2, date_3]

ts_arr = np.stack(list_src_arr, axis=0)

print(ts_arr)
print(ts_arr.shape)

list_dates = ['20180101','20180107','20180115']


[[[ 1.  2.  3.]
  [ 4.  5. nan]
  [ 3.  7.  9.]]

 [[nan  2.  3.]
  [ 4.  5. 10.]
  [nan  7.  9.]]

 [[ 3.  2.  3.]
  [ 9. 10. nan]
  [ 5.  7.  9.]]]
(3, 3, 3)


In [40]:
#for i in range(ts_arr.shape[0]):
#    for j in range(ts_arr.shape[1]):

for i in range(3):
    for j in range(3):
        ts_vec = ts_arr[:,i,j]

        data_dict = {'date':list_dates, 'value':ts_vec}
        
        df = pd.DataFrame.from_dict(data_dict)

        df['date'] = pd.to_datetime(df['date'], format='%Y%m%d')

        df = df.set_index('date')
        
        display(df)

        df_interp = df.resample('10D').mean().interpolate()
        
        display(df_interp)

        print('------------------')

,value
date,
2018-01-01,1.0
2018-01-07,NaN
2018-01-15,3.0


,value
date,
2018-01-01,1.0
2018-01-11,3.0


------------------


,value
date,
2018-01-01,2.0
2018-01-07,2.0
2018-01-15,2.0


,value
date,
2018-01-01,2.0
2018-01-11,2.0


...

,value
date,
2018-01-01,7.0
2018-01-11,7.0


------------------


,value
date,
2018-01-01,9.0
2018-01-07,9.0
2018-01-15,9.0


,value
date,
2018-01-01,9.0
2018-01-11,9.0


------------------


Il faut tout d'abord faire une interpolation linéaire avec la fonction Pandas resample() (attention à bien définir une colonne avec un format Datetime.datetime). Ensuite, les valeurs manquantes sont définies comme poids = 0 dans le vecteur de poids pour le whittaker. Le résultat est obtenu dans le vecteur z pour les dates du vecteur x.

Voici la fonction Whittaker en python. A appliquer sur une série temporelle de reflectance y avec une vecteur de poids w et un lambda = 6 (valeur empirique communiquée par Tom).

In [ ]:
def whitsmw(y, w, lmda): # rem: 2nd order
    # THIS FUNCTION DOES NOT SUPPORT NaN -> USE WEIGHT = 0 INSTEAD!!!
    m = len(y)
    E = sp.sparse.diags(w, 0)
    D = sp.sparse.diags([1,-2,1],[0,1,2], shape=(m-2, m), format="csr")
    z = sp.sparse.linalg.cg(E + lmda * (D.transpose()).dot(D), y * w)
    # remove leading zeros?
    len_z = len(z[0])
    new_len = len(np.trim_zeros(w,'f'))
    diff = len_z-new_len
    if diff>0:
        z[0][:diff] = np.nan
    return z[0]

In [ ]:
## Linear interpolation
dti_val = df[['date','median']].set_index('date',drop=True)
upsampled = dti_val.resample('5D').mean() 
## Whittaker
# Create weights
w = np.array((~upsampled['median'].isna())*1,dtype='double')
upsampled.loc[upsampled['median'].isna(),'median'] = 0 
# Apply filter
z = whitsmw(upsampled['median'].values, w, 6)
x = upsampled.index

In [41]:
df = df.reset_index()

df_interp = df_interp.reset_index()

In [42]:
fig = px.scatter(df, x='date', y="value")
fig.show()

In [43]:
fig1 = px.line(df_interp, x='date', y="value")

#fig.update_traces(line=dict(width=1.5))

#fig.add_trace(go.Scatter(x=df_interp['date'],
#                         y=df_interp['value'],
#                         mode='markers'))

fig2 = px.scatter(df_interp, x='date', y="value")


fig3 = go.Figure(data=fig1.data + fig2.data)


fig3.show()